## 1. Load libraries
This loads the required R packages for:
- **CellTrek** (mapping scRNA → spatial)
- **Seurat** (single-cell integration)
- **CellChat** (cell–cell communication, though not used later in this script)
- Visualization (`ggplot2`, `pheatmap`, `patchwork`, etc.)
- Clustering/statistics (`ConsensusClusterPlus`, `RColorBrewer`).

Need to install this too in env beforehand
#mamba install -c conda-forge -c bioconda bioconductor-singlecellexperiment


In [ ]:
#remove.packages(c("Seurat","SeuratObject","Matrix"))
#install.packages("remotes")
#remotes::install_version("Matrix", version = "1.6.3")   
#remotes::install_version("SeuratObject", version = "4.1.3", dependencies = FALSE)
#remotes::install_version("Seurat", version = "4.3.0") #BUT MAKE SURE YOU SELECT NONE WHEN IT ASK WHICH PACKAGES TP UPDATE/UPGRADE

library(CellTrek)
library(dplyr)
library(Seurat)
#library(SeuratDisk)
library(viridis)
library(ConsensusClusterPlus)
library(RColorBrewer)
library(ggplot2)
library(ggpubr)
library(pheatmap)
library(CellChat)
library(patchwork)
library(sceasy)
library(future)
library(future.apply)

packageVersion("SeuratObject")  # 4.1.3
packageVersion("Seurat")        # 4.3.0
packageVersion("Matrix")  # 1.6.3

In [ ]:
#to clear up session at any point
list=ls()
list
rm(list); gc()
#gc()
#graphics.off()
#closeAllConnections()


## 2. Load input data
Loads the **Visium spatial transcriptomics** object and the **scRNA-seq** Seurat object. In this script:
- `ovVis` = Visium data (early-gen, spot-based)
- `ovSc` = single-cell RNA-seq data (annotated with cell types).

Each of the datasets have already been filtered and subset to contain only spots/cells or genes that pass the filters. These subset versions of the datasets are loaded and used here but we use the raw counts (for the cells/spots remaining) for the workflow as required by CellTrek.

In [ ]:
#cluster
cell2loc = '/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'


ovVis <- readRDS(paste0(cell2loc, "mito_norm_emily.rds"))
ovSc <- readRDS(paste0(cell2loc, "upk10_sc_400_newlyAnnot.rds"))


In [20]:

#local
cell2loc = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'
ov_visium = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'

#### Load Visium Data
ovVis <- readRDS(paste0(ov_visium, "/mito_norm_emily.rds"))  #version of Ollie's filtered against mine
ovVisollie <- readRDS(paste0(ov_visium, "/mito_norm.rds")) 
#ovVis <- readRDS("/home/lythgo02/Documents/OV_visium/final_adata_vis_clean_seurat.rds")

#### Load scRNA-seq
#ovSc <- readRDS("/home/lythgo02/Documents/OV_visium/upk10_sc_400_newlyAnnot.rds")



In [ ]:
ov1 <- ovVis[[1]]
colnames(ov1@meta.data)
slotNames(ov1[["Spatial"]])

lapply(ovVis, function(sample){
    dim(sample[["Spatial"]]@counts)
})


[1] "orig.ident"       "nCount_Spatial"   "nFeature_Spatial" "nCount_SCT"      
[5] "nFeature_SCT"     "SCT_snn_res.0.8"  "seurat_clusters"  "percent_mito"

[1] "counts"        "data"          "scale.data"    "key"          
[5] "assay.orig"    "var.features"  "meta.features" "misc"

$OV_1
[1] 12662   960

$OV_2
[1] 12752  1472

$OV_3
[1] 12718  1315

$OV_4
[1] 12340   573

$OV_5
[1] 12267   777

$OV_6
[1] 12235   681

$OV_1
[1] 32291  1006

$OV_2
[1] 32291  1492

$OV_3
[1] 32291  1454

$OV_4
[1] 32291   760

$OV_5
[1] 32291   808

$OV_6
[1] 32291   753

$OV_7
[1] 32291  2162

$OV_8
[1] 32291   686

$OV_9
[1] 32291  1035

$OV_10
[1] 32291  1020

$OV_11
[1] 32291  1348

$OV_12
[1] 32291  1986

If using my rds list the cell below is a method to convert the images slot to an actual SeuratImage

In [ ]:
ovVis <- readRDS("/home/lythgo02/Documents/OV_visium/split_by_sample/rds/seurat_list_all.rds")

if (!methods::isClass("SeuratImage")) {
  methods::setClass(
    "SeuratImage",
    slots = c(
      image         = "ANY",
      scale.factors = "list",
      coordinates   = "data.frame",
      assay         = "character",
      key           = "character"
    )
  )
}

for (smp in names(ovVis)) {
  obj      <- ovVis[[smp]]
  coords   <- obj@misc$spatial_coords[[smp]]
  img_info <- obj@misc$spatial_images[[smp]]

  # coords -> Seurat style, aligned to cells
  colnames(coords)[1:2] <- c("imagecol","imagerow")
  coords <- coords[match(colnames(obj), rownames(coords)), , drop = FALSE]
  rownames(coords) <- colnames(obj)

  # build minimal SeuratImage S4 (class defined earlier)
  simg <- methods::new("SeuratImage")
  simg@image         <- as.array(img_info$image)      # or img_info$images$hires
  simg@scale.factors <- img_info$scalef               # or img_info$scale.factors
  simg@coordinates   <- coords
  simg@assay         <- DefaultAssay(obj)
  simg@key           <- paste0(smp, "_")

  # >>> attach directly to the images slot (avoid [[<-)
  if (is.null(obj@images)) obj@images <- list()
  obj@images[[smp]] <- simg

  ovVis[[smp]] <- obj
}

# sanity
lapply(ovVis, function(o) class(o@images[[1]]))  # should print "SeuratImage"
# 1) Make sure each spatial object uses RNA as its default assay

for (i in names(ovVis)){
    obj <- ovVis[[i]]
    obj[["RNA"]] <- obj[["originalexp"]]
    DefaultAssay(obj) <- "RNA"
    obj[["originalexp"]] <- NULL
    ovVis[[i]] <- obj
}
ovVis <- lapply(ovVis, function(o) { DefaultAssay(o) <- "RNA"; o })

# 2) If you created SeuratImage manually, ensure its assay slot is "RNA"
for (smp in names(ovVis)) {
  im <- ovVis[[smp]]@images[[1]]
  if (!is.null(im)) im@assay <- "RNA"
  ovVis[[smp]]@images[[1]] <- im
}


Need to rename assays, note when converted to seurat object earlier (code in markdown cell run in r_spatial env) the "raw_counts" and "X" is stored under "originalexp" in the seurat objects 

For spatial we did:   

seurat_obj <- as.Seurat(
            adata,              # your SingleCellExperiment object
            counts = "raw_counts", # assay to use as raw counts
            data = "X" # optional: assay to use as normalized data
)
3. Extract sample per spot
sample <- colData(adata)$sample
seurat_obj@meta.data$sample <- sample

4. Extract coordinates and image info
coords <- reducedDim(adata, "spatial")
coords_list <- split(as.data.frame(coords), sample)
image_list <- metadata(adata)$spatial


5. Assign images and coordinates to Seurat in a CellTrek-compatible way
for (sample_name in names(coords_list)) {
  sp_coords <- coords_list[[sample_name]]
  img_info <- image_list[[sample_name]]

  seurat_obj@images[[sample_name]] <- list(
    image = as.array(img_info$image),
    scale.factors = img_info$scalef,
    coordinates = sp_coords
  )
}
1. Store original coordinates for reference
seurat_obj@misc$spatial_coords <- coords_list
seurat_obj@misc$spatial_images <- image_list

For single cell we did:  

seurat_obj <- as.Seurat(
       singlecell,              
       your SingleCellExperiment object#  counts = "counts", # assay to use as raw counts
       data = "log_counts" # optional: assay to use as normalized data
)


so:  

ovVis[["originalexp"]]$counts # = raw_counts 
ovVis[["originalexp"]]$data #= X

ovSc[["originalexp"]]$counts # = counts 
ovSc[["originalexp"]]$data #= log_counts

But mine lacks the imaging info stored in correct format so filtering Ollie's against mine below

The following silenced chunks are for filtering Ollie's mito_nrom.rds by subsetting to match the colnames (spot IDs) and rownames (genes) present in my rds
 - can't convert the adata to rds in the correct format so taking format from ollie's with processing from mine 
 - saved the end product of subsetting as mito_nrom_emily.rds and that is now loaded at the beginning 

In [ ]:
#ovVis <- readRDS(paste0(ov_visium, "/mito_norm_emily.rds"))

#emily_list <- SplitObject(ovVis_emily, split.by = "sample")
#cells_ollie <- lapply(ovVis_ollie, colnames)

# Get barcodes from each Seurat object in em_list
#cells_emily <- lapply(emily_list, colnames)

#head(cells_emily[[1]])
#head(cells_ollie[[1]])
#need to append sample name to barcodes in Ollie spatial data 

# (Optional) make a backup first
#ovVis_ollie_orig <- ovVis_ollie



Need to append sample name to the colnames/spot barcodes (id) in Ollie's rds to match the formatting of mine

In [ ]:
#for (i in names(ovVis_ollie)){
#    obj <- ovVis_ollie[[i]]  #overwrite names
#    colnames(obj) <- paste0(i, "_", colnames(obj))
#    ovVis_ollie[[i]] <- obj #save back to list
#}
# Check one sample to confirm
#head(colnames(ovVis_ollie[["OV_1"]]))

# 2) Match sample names between ollie list and split emily
#common_samples <- intersect(names(ovVis_ollie), names(emily_list))

#common_samples


Now filter Ollie's spatial data by the list of spot id and genes present in my sample 
 - to ensure the filtering applied is consisted accross workflows worked on by me
 - spots are filtered to match exactly
 - genes are filtered to approximately match, exact numbers may differ due to naming conventions but assuming largely correct 

In [ ]:
#ovVis_ollie_filtered <- lapply(common_samples, function(i) {
#  obj <- DietSeurat(ovVis_ollie[[i]], graphs = TRUE) # remove the NN graphs ollie previously generated as they can disrupt subsetting
#  # --- cells ---
#  emily_barcodes <- colnames(emily_list[[i]])  #get list of spot ids from my rds (which has filtered out low quality spots)
#  cells_keep <- intersect(colnames(obj), emily_barcodes) #keep corresponding cells in Ollie's object 
#  obj <- subset(obj, cells = cells_keep)
#  # --- genes ---
#  emily_genes <- rownames(emily_list[[i]])  #get gene names remaining after filtering 
#  genes_keep <- intersect(rownames(obj), emily_genes)
  obj <- subset(obj, features = genes_keep)
#})

#names(ovVis_ollie_filtered) <- common_samples

#sapply(ovVis_ollie_filtered, ncol)
#sapply(emily_list, ncol)
#sapply(ovVis_ollie_filtered, nrow)
#sapply(emily_list, nrow)

#saveRDS(ovVis_ollie_filtered, paste0(ov_visium, "/mito_norm_emily.rds")


In [ ]:
# Copy originalexp → RNA and set RNA as default
ovSc[["RNA"]] <- ovSc[["originalexp"]] #copy
DefaultAssay(ovSc) <- "RNA" #set to default
ovSc[["originalexp"]] <- NULL  # Remove the old assay

#not needed unless using my rds
#for (i in names(ovVis)){
#    obj <- ovVis[[i]]
#    obj[["RNA"]] <- obj[["originalexp"]]
#    DefaultAssay(obj) <- "RNA"
#    obj[["originalexp"]] <- NULL
#    ovVis[[i]] <- obj
#}



Note first had to convert the outputs from cell2location workflow to rds objects:
 - activate r environment 
   - mamba activate r_spatial
   - open R
   - 
library(zellkonverter)
adata <- readH5AD("/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/final_adata_vis_clean.h5ad")
# Convert SCE to Seurat
seurat_obj <- as.Seurat(
  adata,              # your Spatial/singleCellExperiment object
  counts = "raw_counts", # assay to use as raw counts
  data = "X" # make sure to use X as this contains filtered data 
)
saveRDS(adata, file="/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/final_adata_vis_clean.rds")

#same with single cell data
#write to local directory if it doesn't allow you to write to cluster (h5ad files can be tricky)

In [ ]:
#check if integrated

colnames(ovSc@meta.data)

#sample names for single cell 
unique(ovSc@meta.data$orig.ident)
ovSc@active.assay 

unique(ovSc$updated_annotation)



In [ ]:
#spatial info stored in images slot 
lapply(ovVis, slotNames)

## 3. Integrate scRNA-seq samples
Here multiple scRNA samples are merged and integrated using Seurat's anchor-based pipeline. This corrects for batch effects and aligns datasets into a shared space.
 - SelectIntegrationFeatures finds highly variable genes shared across all samples which are used as anchors for sample integration. 
 - Anchors are pairs of cells across datasets that are similar in expression.
 - object.list = ovSc tells Seurat which samples to integrate.
 - anchor.features = features uses the previously selected genes.

In [ ]:

#already have single object with multiple samples 
# Split the Seurat object by sample and store in list of invdividual samples
ovSc.list <- SplitObject(ovSc, split.by = "orig.ident")

# Check the names
names(ovSc.list)



Single cell is currently previous version 3/4 of suerat object which uses slots, need to convert to version 5 which uses layers 
 - define function to convert to assay5

In [ ]:
class(ovSc.list[[1]][["RNA"]])
sapply(ovSc.list, function(obj) class(obj[["RNA"]]))
#shows that each one of the RNA assays is in old Seurat v3/v4 style (slot based)
#have got code to update to v5 in emails in needed


In [ ]:

#### Iterate over samples in list to prepare for integration

# Make sure every object has HVGs before integration
ovSc.list <- lapply(ovSc.list, function(x){
  DefaultAssay(x) <- "RNA"
  # We already have 'counts' + 'data' layers from earlier conversion
  # Find HVGs on log-normalised data
  x <- NormalizeData(x)
  x <- FindVariableFeatures(
    x,                # use log-normalized values, not raw counts
    selection.method = "vst",
    nfeatures = 3000
  )
  x
})



In [ ]:

# Select features and integrate
features <- SelectIntegrationFeatures(object.list = ovSc.list)

In [ ]:
anchors <- FindIntegrationAnchors(object.list = ovSc.list, 
    normalization.method = "LogNormalize",
    anchor.features = features)

In [ ]:
seuCombined <- IntegrateData(anchorset = anchors)

## 4. Subset to cell types of interest
Selects specific immune/stromal/tumor cell types from the integrated scRNA data.
Not subsetting because already filtered the dataset in cell2location pipeline to exclude those we aren't interested in. 

In [ ]:

#### Subset single cell rna-seq to cell types of interest
unique(seuCombined$updated_annotation)
table(seuCombined$updated_annotation)

#currently not removing any cell types
seuSub <- seuCombined %>% subset(updated_annotation %in% c('Tumour 1','Macrophage','NK_cell','Treg','Tumour 2',
                                                            'Plasma Cells','CD4','CD8','B naive activated','B naive resting','Mesenchymal', 'Monocytes',
                                                            'Dendritic Cells','Neutrophils','Th1','Th17','Endothelial','Tumour 3','Cycling Plasma Cells'))
                                                            
table(seuSub$updated_annotation)


## 5. Co-embed Visium with scRNA and run CellTrek
Uses `CellTrek::traint()` trains a mapping model between single cell profiles and ST data 
 - project Visium spots and scRNA-seq cells into a **shared latent space**.
 - first need to handle images as the split object function (from downgraded seurat versions used here) can't handle images

In [ ]:
class(ovVis)
names(ovVis)

for (i in seq_along(ovVis)){
    name <- names(ovVis)[i]
    obj <- ovVis[[i]]
    assign(name, obj, envir=.GlobalEnv)
    
}



In [ ]:
#checking if colnames of spots are clean 
lapply(ovVis, function(x) any(duplicated(colnames(x))))

In [ ]:

# clean up names 
# make.names() ensures all cell names are valid R names (hanldes special characters)
# RenameCells() applies cleaned names to the Seurat object 
# Clean Visium spot names

#no need to apply to visium, already clean and characters are appropriate 
#ovVis <- lapply(ovVis, function(x){
#    RenameCells(x, new.names = make.names(Cells(x)))
#})

# Clean single-cell names
seuSub <- RenameCells(seuSub, new.names = make.names(Cells(seuSub)))


In [ ]:
seuSub@active.assay
seuSub@assays$RNA

CellTrek uses a random forest regressor to learn the mapping between your single cell reference data and spatial transcriptomics data 
 - traint() trains a mapping model between your single-cell reference and spatial sample
 - The object returned (ovTraintAll[[i]]) contains:
 - Model parameters
 - Fitted distances between spots and reference cells


In [12]:
class(OV_1@images[[1]])
slotNames(OV_1@assays)

ERROR: Error: object 'OV_1' not found


In [ ]:
#try on one sample first

ovtrain_ov1 <- CellTrek::traint(
    st_data    = OV_1,               # spatial Seurat object
        sc_data    = seuSub,                # single-cell reference
        sc_assay   = "RNA",         # exact assay name in seuSub
        cell_names = "updated_annotation"  # column with cell type labels
    )

In [ ]:

ovTraintAll <- lapply(ovVis, function(i) {
    ovTraint <- CellTrek::traint(st_data = i,
                                 sc_data = seuSub,
                                 sc_assay = "RNA",
                                 cell_names = "updated_annotation")
    return(ovTraint)
})

# Add names so they match the input list
names(ovTraintAll) <- names(ovVis)



In [ ]:
#saveRDS(ovTraintAll, paste0(ov_visium, "cellTrek/ovTrainAll_obj.rds"))

cell2loc = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'
ov_visium = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'

ovTraintAll <- readRDS(paste0(ov_visium, "cellTrek/ovTrainAll_obj.rds"))

#cluster
#saveRDS(ovTraintAll, paste0(cell2loc, "/cellTrek/ovTrainAll_obj.rds"))
#ovTraintAll <- readRDS(paste0(cell2loc, "cellTrek/ovTrainAll_obj.rds"))

seuSub <- readRDS(paste0(cell2loc, "cellTrek/seuSub.rds"))

In [ ]:
t <- ovTrainAll[[1]]
unique(t@meta.data$id_new)

## 6. Visualize embeddings
Plots the joint embedding to show how Visium and scRNA cells mix, grouped by type or annotation.

In [ ]:
# ---- build a 19-colour palette----
library(pals)
pal.bands(alphabet, alphabet2, cols25, glasbey, kelly, polychrome, 
  stepped, tol, watlington,
  show.names=FALSE)

In [ ]:

#### View co-embedding
pals19 <- glasbey(19)

cols19 <- c(
  "#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00",
  "#FFFF33", "#A65628", "#F781BF", "#999999", "#66C2A5",
  "#FC8D62", "#8DA0CB", "#E78AC3", "#A6D854", "#FFD92F",
  "#E5C494", "#1B9E77", "#D95F02", "#B3B3B3"
)

# Step 1: Generate plots
c2lPlots <- lapply(ovTraintAll, function(obj) {
  DimPlot(obj, group.by = "updated_annotation", cols = cols19)
})

# Step 2: Save plots
for (i in seq_along(c2lPlots)) {
  ggsave(
    filename = paste0(ov_visium, "cellTrek/DimPlot_OV", i, ".pdf"),
    plot = c2lPlots[[i]],
    width = 8,
    height = 6,
    dpi = 300
  )
}


## 7. Map cells into spatial tissue
Runs `CellTrek::celltrek()` to assign scRNA-seq cells into Visium coordinates, producing pseudo-single-cell maps. Parameters control interpolation, number of trees, PCs, distance threshold, etc.

Run using parallelisation with 4 CPUs on the cluster (interactive session) - moved into scripts for slurm submission

sbatch celltrek.sh (which submits run_celltrek.r)  

 - Note, in "Stardist_Cells_Per_Spot.csv" which lists n nuclei per spot barcode for OV_1, avergae number of cells per spot is 17.7
 - Using this to guide the other samples 


In [ ]:


# --- load libraries ---
library(CellTrek)
library(future)
library(future.apply)
library(progressr)
library(Seurat)

# --- parallelization ---
plan(multisession, workers = 6)           # use 6 cores
options(future.globals.maxSize = 20*1024^3)  # 20 GB per worker
handlers("txtprogressbar")  # to monitor progress

# --- logging set up ---
log_file <- "celltrek_progress"
sink(log_file, append=TRUE, split=TRUE)  #redirects console output to log file, split=TRUE keeps output visible in console while also writing to log 

# --- load your data ---
cell2loc = '/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'

ovTraintAll <- readRDS(paste0(cell2loc, "cellTrek/ovTrainAll_obj.rds"))

seuSub <- readRDS(paste0(cell2loc, "cellTrek/seuSub.rds"))

# --- define celltrek wrapper ---
run_celltrek_fast <- function(trained_obj, sc_data, sample_name) {
  message(Sys.time(), "Processing ", sample_name) #adds timestamp to each log entry 
  
  ct_result <- CellTrek::celltrek(
    st_sc_int = trained_obj,
    int_assay = "traint",
    sc_data = sc_data,
    sc_assay = "RNA",
    reduction = "pca",
    intp = TRUE,
    intp_pnt = 500,  #interpolation points use synthetic cell coordinates to smooth mapping between sc and spatial but too many points crowds the space
    intp_lin = FALSE,
    nPCs = 30,
    dist_thresh = 0.55, #Ollie's value but also higher than default 0.4 to permit for tightly packed cells 
    top_spot = 5, #max number of spots a cell can be mapped to 
    spot_n = 12,  #number of cells one spot can contain, lower than ave 17.7 cells per spot (OV_1) 
    repel_r = 60,   #increase both repel_r and reper_iter to spread points and avoid overcrowding (breaking triangulation if you run DT co-localisation later)
    repel_iter = 20,
    keep_model = TRUE,
    ntree=500
  )
  message(Sys.time(), " | Finished CellTrek for: ", sample_name)
  return(ct_result$celltrek)
}

# --- run CellTrek on all samples ---
with_progress({
  p <- progressor(along = ovTraintAll)
  
  celltrek_results <- future_lapply(
    seq_along(ovTraintAll),
    function(i) {
      sample_name <- names(ovTraintAll)[i]
      message(Sys.time(), " | Processing sample: ", sample_name)
      
      result <- run_celltrek_fast(
        trained_obj = ovTraintAll[[i]],
        sc_data = seuSub,
        sample_name = sample_name
      )
      
      saveRDS(result, file = paste0("cellTrek/celltrek_results", sample_name, ".rds"))
      message(Sys.time(), " | Saved result for: ", sample_name)
      
      p()  # update progress bar
      return(result)
    },
    future.stdout = TRUE
  )
})

saveRDS(celltrek_results, file = paste0("cellTrek/celltrek_results.rds"))

    



View co-embeddings:
 - can first merge (which requires normalisation etc again) if you want to compare treated vs untreated in one integrated embedding
 - or keep as separate objects and plot individual (in which case don't use merged rds)

In [1]:
cell2loc = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/scratchc/fmlab/lythgo02/OV_visium/emily/cell2location/'
#ovTrainAll <- readRDS(paste0(cell2loc, "cellTrek/celltrek_results_ntree1000_p500_celltrek.rds"))
ovTrainAll <- readRDS(paste0(cell2loc, "cellTrek/celltrek_results.rds"))
seuSub <- readRDS(paste0(cell2loc, "cellTrek/seuSub.rds"))

In [14]:
seu <- ovTrainAll[[1]]

slotNames(
    seu@assays)

NULL

In [ ]:
#load 

ovCellTrekList <- ovTrainAll

#names(ovCellTrekList) <- names(ovTraintAll)


#convert updated annotation to factor with levels organised in alphabetical order for plotting purposes 
ovCellTrekList <- lapply(ovCellTrekList, function(ovCellTrek){
    ovCellTrek$cell_type <- ifelse(grepl("Tumour", ovCellTrek$updated_annotation), "Tumour",
                                    ovCellTrek$updated_annotation)  
    ovCellTrek$cell_type <- factor(ovCellTrek$cell_type,
                                    levels=sort(unique(ovCellTrek$cell_type)))                                                        
    return(ovCellTrek)
})

Need to merge Th1 and Th17 to T-helper and merge cycling plasma cells with plasma cells as their counts drop below 20 for multiple samples  
Documentaiton advises n>20 for cell types for scoloc analysis  
NK cells and B naive resting have low n in one or two samples but higher in others so leaving  

In [ ]:

ovCellTrekList <- lapply(ovCellTrekList, function(ovCellTrek) {
    cell_type <- as.character(ovCellTrek@meta.data$cell_type)

    #merge categories with consistently low counts across samples
    cell_type[grepl("Th", cell_type)] <- "T-helper"
    cell_type[grepl("Cycling", cell_type)] <- "Plasma Cells"
    # Add to metadata
    ovCellTrek@meta.data$cell_type_collapsed <- factor(cell_type)

    return(ovCellTrek)
})

In [ ]:
# For each sample, get count table with 'cell_type' and renamed count column
count_tables <- lapply(names(ovCellTrekList), function(sampN) {
  ovCellTrek <- ovCellTrekList[[sampN]]
  
  as.data.frame(ovCellTrek$cell_type_collapsed) %>%
    setNames("cell_type") %>%
    count(cell_type) %>%
    rename(!!sampN := n)  # Rename 'n' to sample name dynamically
})

# Join all tables by 'cell_type'
cell_type_counts_wide <- Reduce(function(x, y) full_join(x, y, by = "cell_type"), count_tables)

# Replace NAs with 0 (if any cell types missing in some samples)
cell_type_counts_wide[is.na(cell_type_counts_wide)] <- 0

# View result
print(cell_type_counts_wide)

ov_visium = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'

write.csv(cell_type_counts_wide, paste0(ov_visium, "/cellTrek/cell_type_counts_collapsed.csv"), row.names = FALSE)

In [ ]:
library(dplyr)
library(tidyr)
library(ggplot2)
# Create palette
cols17_soft <- c(
  "#E9967A",  # Light salmon
  "#87CEFA",  # Light sky blue
  "#90EE90",  # Light green
  "#DDA0DD",  # Plum
  "#FFD700",  # Gold
  "#FFA07A",  # Light coral
  "#B0C4DE",  # Light steel blue
  "#98FB98",  # Pale green
  "#FFB6C1",  # Light pink
  "#AFEEEE",  # Pale turquoise
  "#D8BFD8",  # Thistle
  "#F0E68C",  # Khaki
  "#E6E6FA",  # Lavender
  "#F08080",  # Light red
  "#ADD8E6",  # Light blue
  "#BA55D3",  # Medium orchid 
  "#20B2AA"   # Light sea green 
)




# Create named vector to ensure colors are mapped to cell types consistently
cell_types <- unique(cell_type_props$cell_type)
cell_type_colors <- setNames(cols17_soft[1:length(cell_types)], cell_types)

# Plot
ggplot(cell_type_props, aes(x = sample, y = proportion, fill = cell_type)) +
  geom_bar(stat = "identity", position = "stack") +
  scale_fill_manual(values = cell_type_colors) +
  scale_y_continuous(labels = scales::percent_format()) +
  theme_minimal() +
  labs(
    title = "Proportion of Cell Types per Sample (Collapsed)",
    x = "Sample",
    y = "Proportion of Cells",
    fill = "Cell Type"
  )
  # Save the last plot as PNG
ggsave(
  filename = paste0(ov_visium, "cellTrek/cell_types_plot.png"),
  width = 8,
  height = 5,
  dpi = 300
)




Set x and y limits for plotting 
 - setting coordinates for xlim() and ylim() for ggplot to zoom/crop the plot to specific region of interest

In [ ]:
xl <- list()
xl$OV_1 <- c(15, 585)
xl$OV_2 <- c(20, 580)
xl$OV_3 <- c(20, 580)
xl$OV_4 <- c(20, 580)
xl$OV_5 <- c(20, 580)
xl$OV_6 <- c(15, 585)
xl$OV_7 <- c(15, 585)
xl$OV_8 <- c(15, 585)
xl$OV_9 <- c(25, 575)
xl$OV_10 <- c(20, 575)
xl$OV_11 <- c(25, 575)
xl$OV_12 <- c(25, 575)

names(xl) <- names(ovCellTrekList)


yl <- list()
yl$OV_1 <- c(15, 585)
yl$OV_2 <- c(15, 575)
yl$OV_3 <- c(25, 565)
yl$OV_4 <- c(25, 565)
yl$OV_5 <- c(20, 580)    
yl$OV_6 <- c(20, 570)
yl$OV_7 <- c(35, 560)
yl$OV_8 <- c(15, 585)
yl$OV_9 <- c(25, 575)
yl$OV_10 <- c(5, 575)  
yl$OV_11 <- c(25, 565)
yl$OV_12 <- c(25, 565)

names(yl) <- names(ovCellTrekList)

Chunk below is for plotting cell_types with Th1 and Th17 as well as Cycling Plasma Cells included

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15, repr.plot.res = 150)

cols19 <- c(
  "#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00",
  "#FFFF33", "#A65628", "#F781BF", "#999999", "#66C2A5",
  "#FC8D62", "#8DA0CB", "#A6D854", 
  "#E5C494", "#1B9E77", "#D95F02", "#B3B3B3"
)
cellTrekPlots <- lapply(names(ovCellTrekList), function(sampN){
    ovCellTrek <- ovCellTrekList[[sampN]]

    img_fact <- ovCellTrek@images$slice1@scale.factors$lowres   #scale coordinates to match resolution of histo images 
    img_temp <- ovCellTrek@images$slice1@image                  #pull out image to use as backgroud
    img_data <- ovCellTrek@meta.data %>% dplyr::select(coord_x, coord_y, cell_type:id_new)   #select coordinates and cell types for plotting 
    img_data$coord_x_new=img_data$coord_y*img_fact           #scales coordinates to match image resolution - img_fact is the scaling factor to convert spot coordinates into pixel units.
    img_data$coord_y_new=dim(img_temp)[1]-img_data$coord_x*img_fact #flips axis vertically (visium images are stored with the origin (0,0) at the top-left corner, but spatial coordinates often assume the origin is bottom-left so flip)
    img_data$color_var <- factor(img_data[, "cell_type"])
    img_data$shape_var <- ''

    ggplot(img_data, aes(x=coord_x_new, y=coord_y_new, fill=cell_type)) +
    background_image(raster.img = img_temp)+
    geom_jitter(size = 1, shape = 21, stroke = 0.2, color = "black") +
    scale_fill_manual(values = cols19) +
    xlim(xl[[sampN]]) +
    ylim(yl[[sampN]]) + 
    coord_fixed(ratio = 1)

})

names(cellTrekPlots) <- names(ovCellTrekList)

In [ ]:
cellTrekPlots

Chunk below is for cell types collapsed

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 15, repr.plot.res = 150)

cols19 <- c(
  "#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00",
  "#FFFF33", "#F781BF", "#999999", "#66C2A5",
  "#FC8D62", "#8DA0CB", "#E78AC3", "#A6D854", 
  "#E5C494", "#1B9E77"  
)
cellTrekPlots <- lapply(names(ovCellTrekList), function(sampN){
    ovCellTrek <- ovCellTrekList[[sampN]]

    img_fact <- ovCellTrek@images$slice1@scale.factors$lowres   #scale coordinates to match resolution of histo images 
    img_temp <- ovCellTrek@images$slice1@image                  #pull out image to use as backgroud
    img_data <- ovCellTrek@meta.data %>% dplyr::select(coord_x, coord_y, cell_type_collapsed:id_new)   #select coordinates and cell types for plotting 
    img_data$coord_x_new=img_data$coord_y*img_fact           #scales coordinates to match image resolution - img_fact is the scaling factor to convert spot coordinates into pixel units.
    img_data$coord_y_new=dim(img_temp)[1]-img_data$coord_x*img_fact #flips axis vertically (visium images are stored with the origin (0,0) at the top-left corner, but spatial coordinates often assume the origin is bottom-left so flip)
    img_data$color_var <- factor(img_data[, "cell_type_collapsed"])
    img_data$shape_var <- ''

    ggplot(img_data, aes(x=coord_x_new, y=coord_y_new, fill=cell_type_collapsed)) +
    background_image(raster.img = img_temp)+
    geom_jitter(size = 1, shape = 21, stroke = 0.2, color = "black") +
    scale_fill_manual(values = cols19) +
    xlim(xl[[sampN]]) +
    ylim(yl[[sampN]]) + 
    coord_fixed(ratio = 1)

})

names(cellTrekPlots) <- names(ovCellTrekList)

In [ ]:
cellTrekPlots

In [ ]:
for (i in seq_along(cellTrekPlots)) {
  ggsave(
    filename = paste0(ov_visium, "/cellTrek/celltrek_OV_", i, "_trek.pdf"),
    plot = cellTrekPlots[[i]],
    width = 12,
    height = 9,
    dpi = 300
  )
}



Co-localisation of tumour and Ccl5 expression

In [ ]:
cellTrekPlots <- lapply(names(ovCellTrekList), function(sampN){
  ovCellTrek <- ovCellTrekList[[sampN]]

  img_fact <- ovCellTrek@images$slice1@scale.factors$lowres
  img_temp <- ovCellTrek@images$slice1@image

  # Prepare metadata
  img_data <- ovCellTrek@meta.data %>%
    dplyr::select(coord_x, coord_y, cell_type:id_new) %>%
    mutate(
      CCL5_expr = FetchData(ovCellTrek, vars = "Ccl5")[,1],
      coord_x_new = coord_y * img_fact,
      coord_y_new = dim(img_temp)[1] - coord_x * img_fact,
      
      # NEW: simplified tumor annotation
      outline_group = ifelse(cell_type %in% "Tumour","Tumour","Non-tumour" )
     # outline_group = ifelse(cell_type %in% c("Tumour 1", "Tumour 2", "Tumour 3"), "Tumour", "Non-tumour")
    )

  # Plot
  ggplot(img_data, aes(x = coord_x_new, y = coord_y_new)) +
    background_image(raster.img = img_temp) +
    geom_jitter(
      aes(fill = CCL5_expr, color = outline_group),  # NEW: use simplified group
      shape = 21, size = 1, stroke = 0.2
    ) +
    scale_fill_gradient(low = "white", high = "red", name = "Ccl5 Expression") +
    
    # NEW: custom color scale for simplified groups
    scale_color_manual(
      values = c("Tumour" = "blue", "Non-tumour" = "green"),
      name = "Region"  # optional custom legend title
    ) +
    
    xlim(xl[[sampN]]) +
    ylim(yl[[sampN]]) +
    coord_fixed(ratio = 1) +
    labs(title = paste("Sample:", sampN))
})

cellTrekPlots


In [ ]:
for (i in seq_along(cellTrekPlots)) {
  ggsave(
    filename = paste0(ov_visium, "/cellTrek/celltrek_ccl5_OV_", i, ".pdf"),
    plot = cellTrekPlots[[i]],
    width = 12,
    height = 9,
    dpi = 300
  )
}


Plotting if Ccl5+ tumour, Ccl5+ NK cell or Ccr5+ CD8 cell
 - (each of Ccr1/3/5 receptor bind the Ccl5 chemokine on tumours)

Ccl5 = chemokine involved in recruiting immune cells eg T-cells, NK cells, macrophages and DCs via interaction with receptors like Ccr1, Ccr3 and Ccr5

In [ ]:
ovCellTrekList<- lapply(ovCellTrekList, function(ovCellTrek){
    ccl5_expr <- FetchData(ovCellTrek, vars="Ccl5")[,1] #extract expr of ccl5 as df and convert to vector
    ccr5_expr <- FetchData(ovCellTrek, vars="Ccr5")[,1] #extract expr of ccr5 as df and convert to vector
    ccr1_expr <- FetchData(ovCellTrek, vars="Ccr1")[,1]
    ccr3_expr <- FetchData(ovCellTrek, vars="Ccr3")[,1]

    cell_type <- ovCellTrek@meta.data$cell_type #retrieve cell type label
    coexp_label <- rep("Other", length(ccl5_expr))  #repeat other to create a default label to overwrite
    # Label tumour cells with ccl5 exp
    coexp_label[cell_type=="Tumour" & ccl5_expr >0 ] <- "Ccl5+_Tumour"
    # Logical vector for NK cells positive for any of ccr5, ccr3, or ccr1
    nk_pos <- cell_type == "NK_cell" & (ccr5_expr > 0 | ccr3_expr > 0 | ccr1_expr > 0)
    coexp_label[nk_pos] <- "NK_Ccr_receptor+"
    # Logical vector for CD8 cells positive for any of ccr5, ccr3, or ccr1
    cd8_pos <- cell_type == "CD8" & (ccr5_expr > 0 | ccr3_expr > 0 | ccr1_expr > 0)
    coexp_label[cd8_pos] <- "CD8_Ccr_receptor+"
    # Add to metadata
    ovCellTrek@meta.data$coexp <- coexp_label
    return(ovCellTrek)
})

In [ ]:
lapply(ovCellTrekList, function(x){
    table(x@meta.data$coexp)
})

In [ ]:

coexp_counts_list <- lapply(names(ovCellTrekList), function(sampN) {
  tbl <- table(ovCellTrekList[[sampN]]@meta.data$coexp)
  df <- as.data.frame(tbl)
  colnames(df) <- c("Coexp_Label", "Count")
  df$Sample <- sampN
  df
})

combined_df <- bind_rows(coexp_counts_list)

write.csv(combined_df, paste0(ov_visium,"cellTrek/combined_coexp_counts.csv"), row.names = FALSE)


In [ ]:
cellTrekPlots <- lapply(names(ovCellTrekList), function(sampN) {
  ovCellTrek <- ovCellTrekList[[sampN]]

  img_fact <- ovCellTrek@images$slice1@scale.factors$lowres
  img_temp <- ovCellTrek@images$slice1@image

  # Prepare metadata
  img_data <- ovCellTrek@meta.data %>%
    dplyr::select(coord_x, coord_y, cell_type:id_new, coexp) %>%  # Include coexp
    mutate(
      coord_x_new = coord_y * img_fact,
      coord_y_new = dim(img_temp)[1] - coord_x * img_fact,
    #  outline_group = ifelse(cell_type == "Tumour", "Tumour", "Non-tumour")
    )

  # Plot
  ggplot(img_data, aes(x = coord_x_new, y = coord_y_new)) +
    background_image(raster.img = img_temp) +
    geom_jitter(
      aes(fill = coexp),
      shape = 21, size = 1, stroke = 0.1
    ) +
    scale_fill_manual(
      values = c(
        "Ccl5+_Tumour" = "red",
        "NK_Ccr_receptor+" = "blue",
        "CD8_Ccr_receptor+" ="#4DAF4A",
        "Other" = "grey80"
      ),
      name = "Co-expression"
    ) +

    xlim(xl[[sampN]]) +
    ylim(yl[[sampN]]) +
    coord_fixed(ratio = 1) +
    labs(title = paste("Sample:", sampN))
})


In [ ]:
cellTrekPlots

In [ ]:
for (i in seq_along(cellTrekPlots)) {
  ggsave(
    filename = paste0(ov_visium, "/cellTrek/celltrek_ccl5_OV_", i, ".pdf"),
    plot = cellTrekPlots[[i]],
    width = 12,
    height = 9,
    dpi = 300
  )
}


Spatial Colocalisation 
 - CellTrek::scoloc() analyzes how different cell types are spatially arranged relative to one another.
 - It takes the mapped cells and then, for each cell type, builds a graph-based model of cell-cell proximity and interaction:
 - use_method = 'DT': uses Delaunay triangulation to define spatial neighborhoods or use KNN for k-nearest neighbours
 - boot_n = 200: performs 200 bootstrap iterations to assess statistical robustness
 - the result tells you about spatial overlap between cell types 


In [ ]:
#quantify how often different cell types neighbour each other
ovSGraphKlList <- lapply(ovCellTrekList, function(ovCellTrek){
    CellTrek::scoloc(
        ovCellTrek,
        col_cell = 'cell_type_collapsed',
        use_method = 'KL',
        boot_n = 200
    )
})


Or for generating the results for colocalisation of Ccr5+ tumour cells with Ccr1/3/5 positive NK or CD8 cells  
 - don't advise because insufficient power due to low n

In [ ]:
#quantify how often different cell types neighbour each other
ovSGraphKlList <- lapply(ovCellTrekList, function(ovCellTrek){
    CellTrek::scoloc(
        ovCellTrek,
        col_cell = 'coexp',
        use_method = 'KL',
        boot_n = 200
    )
})

After building repeated graphs, it takes the minimum spanning tree to ensure connected graph that links all cell types while minimising distance  
It then aggregates these MSTs over bootstraps to produce a consensus matrix of cell-type-by-cell-type adjacency  
 - rows/columns = cell types
 - values = how often cell types are directly connected (0=almost never neighbours, 1=always neighbours)
  
Extract the minimum spanning tree:
  -MST = smallest possible graph connecting all nodes (cell types or spatial clusters) such that every node is connected, total edge weight (distance) is minimised

In [ ]:

ovSGraphKlMstConsList <- lapply(ovSGraphKlList, function(x){
    x$mst_cons
})


In [ ]:
lapply(ovSGraphKlMstConsList, function(mat) rownames(mat))
lapply(ovSGraphKlMstConsList, dim)

Sample 5 and 3 are missing cycling plasma cells row/column from matrix so add one full of 0
 - not if using the collapsed cell types

In [ ]:
#mat <- ovSGraphKlMstConsList[[3]]

# Add a row of zeros
#mat <- rbind(mat, setNames(rep(0, ncol(mat)), colnames(mat)))

# Add a column of zeros
#mat <- cbind(mat, setNames(rep(0, nrow(mat)), 'Cycling.Plasma.Cells'))

# Update row and column names
#rownames(mat)[nrow(mat)] <- 'Cycling.Plasma.Cells'
#colnames(mat)[ncol(mat)] <- 'Cycling.Plasma.Cells'

#mat
# Replace the matrix in the list

#ovSGraphKlMstConsList[[3]] <- mat

In [ ]:
#mat <- ovSGraphKlMstConsList[[5]]

# Add a row of zeros
#mat <- rbind(mat, setNames(rep(0, ncol(mat)), colnames(mat)))

# Add a column of zeros
#mat <- cbind(mat, setNames(rep(0, nrow(mat)), 'Cycling.Plasma.Cells'))

# Update row and column names
#rownames(mat)[nrow(mat)] <- 'Cycling.Plasma.Cells'
#colnames(mat)[ncol(mat)] <- 'Cycling.Plasma.Cells'

#mat
# Replace the matrix in the list

#ovSGraphKlMstConsList[[5]] <- mat

In [ ]:
# Define the desired order (from OV_1)
#desired_order <- c(
  "B.naive.activated","B.naive.resting","CD4","CD8",
  "Cycling.Plasma.Cells","Dendritic.Cells","Endothelial","Macrophage",
  "Mesenchymal","Monocytes","Neutrophils","NK_cell","Plasma.Cells",
  "Th1","Th17","Treg","Tumour"
)

# Manually reorder OV_2
#ovSGraphKlMstConsList$OV_3 <- ovSGraphKlMstConsList$OV_3[desired_order, desired_order]

# Manually reorder OV_3
#ovSGraphKlMstConsList$OV_5 <- ovSGraphKlMstConsList$OV_5[desired_order, desired_order]


In [ ]:
saveRDS(ovSGraphKlMstConsList, paste(ov_visium,"cellTrek/ovSGraphKlMstConsList.rds"))
saveRDS(ovSGraphKlList, paste(ov_visium, "cellTrek/ovSGraphList.rds"))
#ovSGraphKlMstConsList

Visualise:
Compute *average* spatial adjacency matrices treated/untreated
 - ovSGraphKlMstConsList[[i]] is a consensus graph / adjacency matrix derived from CellTrek analysis.
 - Rows and columns: Usually represent cell types or clusters.
 - Values = Strength of interaction, co-localization, or connection probability between two clusters.

Symmetric: A–B = B–A.

So each ovSGraphKlMstConsList[[i]] encodes cluster–cluster connectivity for one sample. - 

In [ ]:

#compute mean pairwise distances between cell types
preTreat <- (ovSGraphKlMstConsList[[1]] + ovSGraphKlMstConsList[[2]] + ovSGraphKlMstConsList[[3]]) / 3
postTreat <- (ovSGraphKlMstConsList[[4]] + ovSGraphKlMstConsList[[6]]+ ovSGraphKlMstConsList[[5]]) / 3

#take the difference
treatDiff <-  postTreat - preTreat

#negative values after post-pre means cells are closer, positive values means cells are further apart 

In [ ]:

  write.csv(preTreat,
    file = paste0(ov_visium, "cellTrek/coloc_matrix_pretreat.csv"),
   # file = paste0(ov_visium, "cellTrek/coloc_matrix_pretreat_ccl5ccr5.csv"),
    row.names = TRUE)


  write.csv(postTreat,
    file = paste0(ov_visium, "cellTrek/coloc_matrix_posttreat.csv"),
   # file = paste0(ov_visium, "cellTrek/coloc_matrix_posttreat_ccl5ccr5.csv"),
    row.names = TRUE)


  write.csv(treatDiff,
    file = paste0(ov_visium, "cellTrek/coloc_matrix_treatdiff.csv"),
    #file = paste0(ov_visium, "cellTrek/coloc_matrix_treatdiff_ccl5ccr5.csv"),
    row.names = TRUE)

Generate heatmaps from co-localisation scores

In [ ]:
#if the option below won't save
preTreat[upper.tri(preTreat)] <- NA
pheatmap(as.matrix(preTreat),
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         border_color ="white",
         na_col="white",
        filename = paste0( ov_visium, "/cellTrek/preTreat_coloc_heatmap.png"))
        #filename = paste0( ov_visium, "/cellTrek/preTreat_coloc_heatmap_ccl5ccr5.png"))

In [ ]:

ov_visium = '/run/user/1804238067/gvfs/sftp:host=clust1-sub-1,user=lythgo02/mnt/nas-data/fmlab/group_folders/lythgo02/OV_visium/emily/cell2location/'

png(paste0(ov_visium, "cellTrek/preTreat_coloc_heatmap.png"), width=800, height=800)
preTreat[upper.tri(preTreat)] <- NA
pheatmap(as.matrix(preTreat),
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         border_color ="white",
         na_col="white")
        # filename = "/home/lythgo02/Documents/OV_visium/cellTrek/preTreat_coloc_heatmap.png"
dev.off()

png(paste0(ov_visium, "cellTrek/postTreat_coloc_heatmap.png"),width=800, height=800)
#png(paste0(ov_visium, "cellTrek/postTreat_coloc_heatmap_ccl5ccr5.png"),width=800, height=800)
postTreat[upper.tri(postTreat)] <- NA
pheatmap(as.matrix(postTreat),
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         border_color ="white",
         na_col="white")
dev.off()

png(paste0(ov_visium, "cellTrek/treatDiff_coloc_heatmap.png"),width=800, height=800)
#png(paste0(ov_visium, "cellTrek/treatDiff_coloc_heatmap_ccl5ccr5.png"),width=800, height=800)
treatDiff[upper.tri(treatDiff)] <- NA
pheatmap(as.matrix(treatDiff),
         cluster_cols = FALSE,
         cluster_rows = FALSE,
         border_color ="white",
         na_col="white", 
        center = 0)
dev.off()



In [ ]:
class(ovSGraphKlMstConsList[[1]])

In [ ]:
# Loop over names and write each data frame to a CSV
lapply(names(ovSGraphKlMstConsList), function(i) {
  write.csv(
    ovSGraphKlMstConsList[[i]],
    file = paste0(ov_visium, "cellTrek/coloc_matrix_", i, ".csv"),
    row.names = FALSE
  )
})


In [ ]:
cellTrekDistPlot <- pheatmap(as.matrix(treatDiff),
                             cluster_cols = FALSE,
                             cluster_rows = FALSE,
                             border_color ="#404040",
                           #  breaks = breaks,
                             na_col="black")

In [16]:
names(ovSGraphKlMstConsList)

ERROR: Error: object 'ovSGraphKlMstConsList' not found


In [ ]:
heatPlots <- lapply(names(ovSGraphKlMstConsList), function(sample_name) {
  x <- ovSGraphKlMstConsList[[sample_name]]
  x[upper.tri(x)] <- NA

  png(paste0(ov_visium, "cellTrek/coloc_heatmap_", sample_name, ".png"), width = 800, height = 800)

  # Force pheatmap to plot inside the png device
  print(pheatmap(as.matrix(x),
                 cluster_cols = FALSE,
                 cluster_rows = FALSE,
                 border_color = "white",
                 na_col = "white"))

  dev.off()
})



In [ ]:
heat_grobs <- lapply(names(ovSGraphKlMstConsList), function(sample_name) {
  x <- ovSGraphKlMstConsList[[sample_name]]
  x[upper.tri(x)] <- NA

  # Generate pheatmap grob
  ph <- pheatmap(
    mat = as.matrix(x),
    cluster_cols = FALSE,
    cluster_rows = FALSE,
    border_color = "white",
    na_col = "white",
    main = sample_name,   # Optional title
    silent = TRUE         # Don't draw, just return grob
  )

  ph$gtable
})


In [ ]:
library(gridExtra)
library(grid)# Display all heatmaps in a 2-column layout (adjust as needed)
grid.arrange(grobs = heat_grobs, ncol = 3)

In [ ]:
#giotto installed in r_spatial env locally to switch to that next?


ERROR: Error: object 'oblabels' not found
